In [1]:
#Question-1
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/USA_Housing (1).csv')

print(df.head())

X = df.drop('Price', axis=1)
y = df['Price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kf = KFold(n_splits=5, shuffle=True, random_state=1)

best_r2 = -np.inf
best_beta = None
r2_scores = []

for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_transpose = X_train.T
    beta = np.linalg.inv(X_train_transpose @ X_train) @ X_train_transpose @ y_train

    y_pred = X_test @ beta

    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    if r2 > best_r2:
        best_r2 = r2
        best_beta = beta

print(f"R2 Scores for each fold: {r2_scores}")
print(f"Best R2 Score: {best_r2}")
print(f"Best Beta (𝛽) matrix: {best_beta}")

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

y_pred_full = X_test_full @ best_beta

final_r2 = r2_score(y_test_full, y_pred_full)

print(f"R2 Score on the test set: {final_r2}")


   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0       79545.45857             5.682861                   7.009188   
1       79248.64245             6.002900                   6.730821   
2       61287.06718             5.865890                   8.512727   
3       63345.24005             7.188236                   5.586729   
4       59982.19723             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population         Price  
0                          4.09      23086.80050  1.059034e+06  
1                          3.09      40173.07217  1.505891e+06  
2                          5.13      36882.15940  1.058988e+06  
3                          3.26      34310.24283  1.260617e+06  
4                          4.23      26354.10947  6.309435e+05  
R2 Scores for each fold: [-10.300071205944167, -11.822298643817328, -12.094531071359691, -11.508307680681673, -10.862223732060572]
Best R2 Score: -10.300071205944167
Best Beta (𝛽) mat

In [2]:
#Question-2
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/USA_Housing (1).csv')
X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.44, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.68, random_state=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

def gradient_descent(X, y, learning_rate, num_iterations):
    m, n = X.shape
    weights = np.zeros(n)
    for i in range(num_iterations):
        y_pred = X @ weights
        gradient = (-2/m) * (X.T @ (y - y_pred))
        weights -= learning_rate * gradient
    return weights

learning_rates = [0.001, 0.01, 0.1, 1]
num_iterations = 1000
best_r2_val = -np.inf
best_weights = None

for lr in learning_rates:
    weights = gradient_descent(X_train_scaled, y_train, learning_rate=lr, num_iterations=num_iterations)
    y_val_pred = X_val_scaled @ weights
    r2_val = r2_score(y_val, y_val_pred)
    y_test_pred = X_test_scaled @ weights
    r2_test = r2_score(y_test, y_test_pred)
    print(f"Learning Rate: {lr}, Validation R²: {r2_val:.4f}, Test R²: {r2_test:.4f}")
    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_weights = weights

print(f"\nBest Regression Coefficients: {best_weights}")
print(f"Best Validation R² Score: {best_r2_val}")


Learning Rate: 0.001, Validation R²: -11.3406, Test R²: -11.1197
Learning Rate: 0.01, Validation R²: -11.2835, Test R²: -11.0665
Learning Rate: 0.1, Validation R²: -11.2835, Test R²: -11.0665
Learning Rate: 1, Validation R²: -inf, Test R²: -inf

Best Regression Coefficients: [227824.57103874 162953.94955334 122053.79216548   2375.65811199
 151147.90687675]
Best Validation R² Score: -11.283521486523076


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1220: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1220: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
